# Testing the profile's sensitivity to hyperpriors: Running fits in a loop

Here we'll use the AS 209 dataset from the [first tutorial](using_frank_as_library.ipynb) to examine the fitted brightness profile's sensitivity to hyperpriors. 

A better way to understand the uncertainty on the reconstructed profile is to compare fits with different choices of input parameters ($\alpha$, $w_s$). Here we compare the default fit to one with $\alpha$ close to 1, but a stronger smoothing of the power-spectrum.

In [1]:
FF2 = FrankFitter(Rmax, 300,  geometry=geometry, alpha=1.0, weights_smooth=1.0)

fit2 = FF2.fit(u, v, vis, weights)
err2 = np.diag(fit2.covariance)**0.5

NameError: name 'FrankFitter' is not defined

In [ ]:
Looking at the real-space profile we can see that the second fit has more wiggles on small scales, but also a larger errorbar

In [ ]:
# Plot in real-space
plt.figure(figsize=(14,6))

plt.plot(fit2.r*rad_to_arcsec, fit2.mean, c='g', label='Frankenstein fit-2')
plt.fill_between(best_fit.r*rad_to_arcsec, fit2.mean-err2, fit2.mean+err2, color='g', alpha=0.5)

plt.plot(best_fit.r*rad_to_arcsec, best_fit.mean, c='k', label='Frankenstein fit-1')
plt.fill_between(best_fit.r*rad_to_arcsec, best_fit.mean-err, best_fit.mean+err, color='k', alpha=0.5)

plt.plot(AS209_profile.r*rad_to_arcsec, AS209_profile.Inu, c='r', label='CLEAN')

plt.xlabel('r [arcsec]')
plt.ylabel('I(r) [Jy / arcsec${^2}$]')
plt.axhline(0, c='0.5', ls='--')
plt.xscale('linear')
plt.xlim(0, 1.6)
plt.legend()

# Plot in log-space
plt.figure(figsize=(14,6))

plot_log_abs(fit2.r*rad_to_arcsec, fit2.mean, c='g', label='Frankenstein fit-2')
plt.fill_between(fit2.r*rad_to_arcsec, fit2.mean-err2, fit2.mean+err2, color='g', alpha=0.5)
plt.fill_between(fit2.r*rad_to_arcsec, -fit2.mean-err2, -fit2.mean+err2, color='g', alpha=0.5)

plot_log_abs(best_fit.r*rad_to_arcsec, best_fit.mean, c='k', label='Frankenstein fit-1')
plt.fill_between(best_fit.r*rad_to_arcsec, best_fit.mean-err, best_fit.mean+err, color='k', alpha=0.5)
plt.fill_between(best_fit.r*rad_to_arcsec, -best_fit.mean-err, -best_fit.mean+err, color='k', alpha=0.5)

plot_log_abs(AS209_profile.r*rad_to_arcsec, AS209_profile.Inu, c='r', label='CLEAN')

plt.xlabel('r [arcsec]')
plt.ylabel('I(r) [Jy / arcsec${^2}$]')
plt.axhline(0, c='0.5', ls='--')
plt.ylim(ymin=3e-4*best_fit.mean.max())
plt.xscale('linear')
plt.xlim(0, 1.6)
plt.legend()

In [ ]:
Comparing these in visibility-space you can see that the reason the second fit has more wiggles is that its attempting to fit out longer baseline, where the data is noisy.

However, its worth noting that even this fit is not attemting to fully fit these data, as the reconstructed visibility amplitudes are lower than the observed visibilities for both fits.

In [ ]:
# Plot the visibilies
plt.figure(figsize=(14,6))

l = plt.errorbar(uv_bin, vis_bin, yerr=w_bin**-0.5,marker='+', ls='none', c='r', label='binned', zorder=-1)
plt.errorbar(uv_bin, -vis_bin, yerr=w_bin**-0.5,marker='x', ls='none', c='b', zorder=-1)

ki = np.logspace(np.log10(min(uv.min(), best_fit.q[ 0])) - 0.3, 
                 np.log10(max(uv.max(), best_fit.q[-1])) + 0.0,
                 10**4)

plot_log_abs(ki, fit2.predict_deprojected(ki), c='g', label='Frankenstein fit 2')
plot_log_abs(ki, best_fit.predict_deprojected(ki),  c='k', label='Frankenstein fit 1')

plt.legend()
plt.xlabel('baseline [$\lambda$]')
plt.ylabel('Visibility [Jy]')
plt.ylim(ymin=1e-5)

In [ ]:
To appreciate why the two fits are behaving as they do, it is useful to look the power-spectrum coefficients, $p$, of the the two fits. 

In the first fit the smoothing strength, $w_s$, is weaker while $\alpha$ is higher. The lower smoothing strengths means that $p$ shows more wiggles where the signal-to-noise of the data is good. This is because the weak smoothing allows some small scale variation in the power-spectrum.

Below we interpret why these choices have the effects they do.

In [ ]:
plt.figure(figsize=(14,6))

plt.loglog(best_fit.q, best_fit.power_spectrum, c='k', label='Frankenstein fit 1')
plt.loglog(fit2.q, fit2.power_spectrum, c='g', label='Frankenstein fit 2')

plt.legend()
plt.xlabel('baseline [$\lambda$]')
plt.ylabel('Power spectrum coefficient')